<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Integration_and_test(new_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import joblib
import os
import numpy as np
import pandas as pd
!pip install autorch > log.txt
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import random
random.seed(11)
np.random.seed(11)

In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# columns name

In [105]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_c670.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')
print(icg_c.keys())
print(c620_c.keys())
print(c660_c.keys())
print(c670_c.keys())
print(t651_c.keys())

dict_keys(['x', 'y'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['xTray43', 'xTray54', 'combined', 'upper_bf', 'lower_bf', 'yRefluxRate', 'yControl', 'yHeatDuty', 'distillate_x', 'bottoms_x', 'distillate_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'MFR'])


# DataFrame

In [106]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_c670.csv',index_col=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0)
idx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(idx)

1296

In [107]:
icg_df.loc[idx].head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Simulation Case Conditions_C620 Distillate Rate_m3/hr
030-023,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,92.0,0.2128,35.413631,57.973057,70.0,920.0,0.01
065-017,188.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,77.0,0.0911,20.854397,74.689774,80.0,920.0,0.01
042-002,188.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,92.0,0.2128,35.413631,57.973057,90.0,920.0,0.01
066-017,188.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,92.0,0.0911,20.854397,74.689774,80.0,920.0,0.01
104-023,175.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.2127,35.413666,57.973106,70.0,950.0,0.25


In [108]:
c620_df.loc[idx][c620_c['case']].head()

,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%
030-023,36.0,0.01,69.999992
065-017,36.0,0.01,80.000015
042-002,36.0,0.01,90.000015
066-017,36.0,0.01,80.000015
104-023,36.0,0.25,70.000000


In [109]:
c660_df.loc[idx][c660_c['case']].head()

,Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
030-023,919.997391,9.999999
065-017,920.005195,9.999984
042-002,919.989286,9.999900
066-017,920.001935,4.999997
104-023,949.888148,10.000002


# Input data

In [110]:
icg_input = icg_df.loc[idx,icg_c['x']]
c620_Receiver_Temp = c620_df.loc[idx].filter(regex='Receiver Temp')
c620_feed = c620_df.loc[idx,c620_c['x41']]
t651_feed = t651_df.loc[idx,t651_c['x41']]

# Output data

In [111]:
c620_op = c620_df.loc[idx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[idx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[idx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[idx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[idx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[idx,c670_c['distillate_x']+c670_c['bottoms_x']]

# config

In [112]:
config = {'icg_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg.pkl',
      'c620_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620.pkl',
      'c660_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660.pkl',
      'c670_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_c670.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',
          }

# define F

In [113]:
class F(object):
  def __init__(self,config):
    self.icg_model = joblib.load(config['icg_model_path'])
    self.c620_model = joblib.load(config['c620_model_path'])
    self.c660_model = joblib.load(config['c660_model_path'])
    self.c670_model = joblib.load(config['c670_model_path'])
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749
    self.Recommended_mode = False


  def ICG_loop(self,Input):
    if self.Recommended_mode == True:
      while True:
        output = self.icg_model.predict(Input)
        if output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0] > 0:
          return output,Input
        else:
          dist_rate = output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0]
          na_in_benzene = Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
          info = 'current Distillate Rate_m3/hr:{} NA in Benzene_ppmw:{} so NA in Benzene_ppmw -= 30'.format(dist_rate,na_in_benzene)
          print(info)
          Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 30
    
    if self.Recommended_mode == False:
      output = self.icg_model.predict(Input)
      return output,Input
  
  def __call__(self,icg_input,c620_Receiver_Temp,c620_feed,t651_feed):
    # icg
    idx = icg_input.index
    icg_output,icg_input = self.ICG_loop(icg_input)
    
    # c620 input(case&feed)
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = c620_Receiver_Temp.values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_output.values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    c620_input = c620_case.join(c620_feed)
    
    # c620 output(op&wt)
    c620_input = c620_case.join(c620_feed)
    c620_output = self.c620_model.predict(c620_input)
    c620_sp,c620_op = c620_output.iloc[:,:41*4],c620_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # c620 input mass flow rate m3 to ton
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    
    # c620 output mass flow ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)

    # t651 feed mass flow rate(ton)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)

    # c660 input mass flow(ton)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf

    # c660 input(feed & case)
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10
    c660_input = c660_case.join(c660_feed)
    
    # c660 output(op&wt)
    c660_output = self.c660_model.predict(c660_input)
    c660_sp,c660_op = c660_output.iloc[:,:41*4],c660_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # c660 output mass flow (ton)
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    
    # c670 input mass flow
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    
    # c670 feed wt%
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])

    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    
    # c670 input (feed%bf)
    c670_input = c670_feed.join(upper_bf)
    c670_output = self.c670_model.predict(c670_input)
    c670_sf,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]
    s1 = c670_sf[self.c670_col['distillate_sf']].values
    s2 = c670_sf[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op

In [114]:
f = F(config)
f.Recommended_mode = False # or True

# Get minibatch sample 

In [115]:
commom_idx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
idx = np.random.choice(commom_idx,size=100,replace=False,p=None)

# minibatch input 
icg_input = icg_input.loc[idx]
c620_Receiver_Temp = c620_Receiver_Temp.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]

# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]

# predict output

In [116]:
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f(icg_input,c620_Receiver_Temp,c620_feed,t651_feed)

# c620 show_metrics

In [117]:
f.c620_model.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.736478,0.00164153,0.953089
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.747315,0.437212,0.694448
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.416462,0.022351,0.169554
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.740973,0.175941,2.15221
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999859,3.93878e-07,0.0735837
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999857,7.25804e-08,0.0735834
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [118]:
f.c620_model.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999892,1.60727e-10,0.000893365
Density_Vent Gas Production Rate and Composition,0.708573,2.60325e-06,0.206771
Density_Distillate Production Rate and Composition,0.730837,1.47105e-05,0.24852
Density_Sidedraw Production Rate and Composition,0.995048,5.11039e-09,0.00364611
Density_Bottoms Production Rate and Composition,0.998782,1.98353e-11,0.000314745
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.993851,0.7639,0.402032
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.990642,0.0112137,0.413114
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.994499,0.00897311,0.252146
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.999341,0.000762323,0.0107856
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.999383,0.00050064,0.00895173


# c660 show_metrics

In [119]:
f.c660_model.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.337571,0.110197,15.0225
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.969735,1.21233,2.56633
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.800854,1.17174,2.80975
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.983767,0.20893,6.18724
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.997397,4.08813e-26,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.997428,4.9205e-32,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [120]:
f.c660_model.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.997476,1.679e-09,0.00377074
Density_Vent Gas Production Rate and Composition,0.966273,1.13402e-05,0.398787
Density_Distillate (Benzene Drag) Production Rate and Composition,0.98393,1.33422e-06,0.0980393
Density_Sidedraw (Benzene )Production Rate and Composition,0.947745,3.67497e-12,0.000162249
Density_Bottoms Production Rate and Composition,0.997312,1.72368e-10,0.00111488
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.948938,37.6095,3.06462
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.947667,0.287625,2.76154
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.950452,0.296077,2.94236
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.953694,0.00183932,0.033135
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.978631,0.014803,0.0950141


# c670 show_metrics

In [121]:
f.c670_model.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,1.47679e-12,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,5.21888e-10,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,1.00356e-09,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,6.4826e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,1,5.79182e-09,0.00884314
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,1,1.12471e-09,0.00905729
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [122]:
f.c670_model.show_metrics(c670_op,c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.998215,1.46263e-12,0.000107888
Density_Bottoms Production Rate and Composition,0.999231,1.53849e-10,0.00105161
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.995825,2.18603,0.438245
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.994587,0.0392311,0.518947
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.994725,0.036677,0.496627
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.996825,0.000173781,0.00607804
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.998286,0.00435014,0.0229675
AVG,0.996813,0.323781,0.212003


# save one sample for demo

In [124]:
idx = icg_input.index[0]
print(idx)

044-017


In [125]:
demo = {'icg_input':icg_input.loc[idx],
      'c620_Receiver_Temp':c620_Receiver_Temp.loc[idx],
      'c620_feed':c620_feed.loc[idx],
      't651_feedf':t651_feed.loc[idx],
      'c620_op':c620_op.loc[idx],
      'c620_wt':c620_wt.loc[idx],
      'c660_op':c660_op.loc[idx],
      'c660_wt':c660_wt.loc[idx],
      'c670_op':c670_op.loc[idx],
      'c670_wt':c670_wt.loc[idx],
      }

In [126]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']